In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# importamos los datos a trabajar para el entrenamiento
data = pd.read_parquet('train.parquet')

In [5]:
# verificamos los valores nulos en el archivo
data.isnull().sum()

id                              0
url                             0
region                          0
region_url                      0
price                           0
type                            0
sqfeet                          0
beds                            0
baths                           0
cats_allowed                    0
dogs_allowed                    0
smoking_allowed                 0
wheelchair_access               0
electric_vehicle_charge         0
comes_furnished                 0
laundry_options             71171
parking_options            126682
image_url                       0
description                     2
lat                          1722
long                         1722
state                           0
dtype: int64

In [4]:
# para no eliminar datos que puedan ser importantes lo cambiamos los archivos none por sin dato en las columnas que tienen el dato none
data['laundry_options'] = data['laundry_options'].replace({None: 'Sin dato'})
data['parking_options'] = data['parking_options'].replace({None: 'Sin dato'})

In [5]:
# verificamos de nuevo el df y verificamos que no hay valores nulos en las columnas a entrenar
data.isnull().sum()

id                            0
url                           0
region                        0
region_url                    0
price                         0
type                          0
sqfeet                        0
beds                          0
baths                         0
cats_allowed                  0
dogs_allowed                  0
smoking_allowed               0
wheelchair_access             0
electric_vehicle_charge       0
comes_furnished               0
laundry_options               0
parking_options               0
image_url                     0
description                   2
lat                        1722
long                       1722
state                         0
dtype: int64

In [6]:
# usamos la columna price para discretizar datos y colocarlos en una lista
data1 = list(data['price'])
category_price = []

for i in data1:
    if i in range(0,1000):
        category_price.append('low')
    elif i in range(1000, 2000):
        category_price.append('medium')
    elif (i >= 2000):
        category_price.append('high')
    else:
        category_price.append(None)

In [7]:
# asignamos la columna category_price y verificamos la presencia de los valores low en la columna
data = data.assign(category_price = category_price)
data['category_price'] = np.where(data['category_price'] == 'low', 1, 0)

In [8]:
# usamos las columnas a transformar para el entrenamiento
new_data = data[['region', 'type', 'sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed', 'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'laundry_options', 'parking_options', 'category_price', 'state', 'url', 'region_url', 'image_url']]

In [9]:
# verificamos los datos a trabajar para transformar
new_data['region'].value_counts()

jacksonville      3849
columbus          3377
rochester         3324
jackson           3306
fayetteville      3295
                  ... 
southwest MS        11
southwest TX         9
st louis             9
fort smith, AR       5
kansas city          3
Name: region, Length: 404, dtype: int64

In [11]:
# importamos el onehotencoder para codificar los datos de tipo objeto en formato binomial
from sklearn.preprocessing import OneHotEncoder
nominal_codificador = OneHotEncoder(sparse_output=False)
nominal_codificador.fit_transform(new_data[['region', 'type', 'laundry_options', 'parking_options', 'state']])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# verificamos las categorias de los datos a transformar
nominal_codificador.categories_

[array(['SF bay area', 'abilene', 'akron / canton', 'albany',
        'albuquerque', 'altoona-johnstown', 'amarillo', 'ames',
        'anchorage / mat-su', 'ann arbor', 'annapolis',
        'appleton-oshkosh-FDL', 'asheville', 'ashtabula', 'athens',
        'atlanta', 'auburn', 'augusta', 'austin', 'bakersfield',
        'baltimore', 'baton rouge', 'battle creek',
        'beaumont / port arthur', 'bellingham', 'bemidji', 'bend',
        'billings', 'binghamton', 'birmingham', 'bismarck', 'bloomington',
        'bloomington-normal', 'boise', 'boone', 'boston', 'boulder',
        'bowling green', 'bozeman', 'brainerd', 'brownsville', 'brunswick',
        'buffalo', 'butte', 'cape cod / islands', 'catskills',
        'cedar rapids', 'central NJ', 'central louisiana',
        'central michigan', 'champaign urbana', 'charleston', 'charlotte',
        'charlottesville', 'chattanooga', 'chautauqua', 'chicago', 'chico',
        'chillicothe', 'cincinnati', 'clarksville', 'cleveland',
        

In [13]:
# importamos las librerias a trabajar para el modelamiento de datos
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [14]:
# verificamos los tipos de datos del nuevo df
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346479 entries, 0 to 346478
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   region                   346479 non-null  object 
 1   type                     346479 non-null  object 
 2   sqfeet                   346479 non-null  int64  
 3   beds                     346479 non-null  int64  
 4   baths                    346479 non-null  float64
 5   cats_allowed             346479 non-null  int64  
 6   dogs_allowed             346479 non-null  int64  
 7   smoking_allowed          346479 non-null  int64  
 8   wheelchair_access        346479 non-null  int64  
 9   electric_vehicle_charge  346479 non-null  int64  
 10  comes_furnished          346479 non-null  int64  
 11  laundry_options          346479 non-null  object 
 12  parking_options          346479 non-null  object 
 13  category_price           346479 non-null  int32  
 14  stat

In [16]:
# Separamos la parte numerica con la categorica
numeric_list = ['sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed','smoking_allowed', 'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished']
categoria_list = ['region', 'type', 'laundry_options', 'parking_options', 'state']

In [17]:
# llamamos la funcion pipeline para datos de tipo numerico y de categoria para transformar
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categoria_transformer =Pipeline(steps=[('onehot', OneHotEncoder())])

In [18]:
# transformamos los datos
transformer = ColumnTransformer([('num', numeric_transformer, numeric_list), ('cat', categoria_transformer, categoria_list)])

In [19]:
# guardamos los datos de caracteristicas a excepcion de la columna a predecir
X = new_data.drop('category_price', axis=1)

In [20]:
# guardamos los datos que queremos predecir despues del testeo
y = new_data['category_price']

In [21]:
# importamos el train_test_split para el entrenamiento de los datos con un 80% de datos como entrenamiento y el 20% como testeo
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8)

In [22]:
# hacemos un shape para ver la cantidad de datos
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((277183, 17), (69296, 17), (277183,), (69296,))

In [23]:
# probamos el modelo de regresion lineal
from sklearn.linear_model import LinearRegression
linear_regresion = LinearRegression()

In [24]:
# importamos los datos y la funcion instancianda
model_linear_regresion = Pipeline([('transformer', transformer), ('Linear_regresion', linear_regresion)])

In [25]:
# corremos el modelo con los datos de entrenamiento
model_linear_regresion.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['sqfeet', 'beds', 'baths',
                                                   'cats_allowed',
                                                   'dogs_allowed',
                                                   'smoking_allowed',
                                                   'wheelchair_access',
                                                   'electric_vehicle_charge',
                                                   'comes_furnished']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['region', 'type',
                                                   'laundry_options',
                                                   'parking_options',
                                                   'state'])])),
                ('Linear_regresion', LinearRegression())])

In [26]:
# corremos la prediccion con el dato de testeo y nos da la prediccion del modelo segun el dato de testeo
y_pred = model_linear_regresion.predict(X_test)
y_pred

array([0.13817093, 0.13814569, 0.82895167, ..., 0.72610004, 0.82337996,
       0.15455372])

In [28]:
# comparamos con los datos de testeo reales con los que nos da el modelo
list(y_test)

[0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,


In [81]:
# evaluamos el modelo usando crossvalidation
from sklearn.model_selection import cross_val_score
linear_scores = cross_val_score(model_linear_regresion, X, y, scoring='neg_mean_squared_error', cv=10)
linear_scores

array([-0.13321787, -0.13325928, -0.13271699, -0.13424384, -0.13245649,
       -0.13214666, -0.13268492, -0.13217224, -0.13331063, -0.13635156])

In [82]:
linear_rmse_scores =np.sqrt(-linear_scores)
linear_rmse_scores

array([0.36499024, 0.36504696, 0.36430343, 0.36639301, 0.36394573,
       0.36351983, 0.36425941, 0.363555  , 0.36511728, 0.36925812])

In [49]:
# creamos una funcion para que nos de los valores de media scores y division standard
def display_scores(scores):
    print('Scores:',scores)
    print('Mean:',scores.mean())
    print('Standard Deviaton:', scores.std())

In [84]:
display_scores(linear_rmse_scores)

Scores: [0.36499024 0.36504696 0.36430343 0.36639301 0.36394573 0.36351983
 0.36425941 0.363555   0.36511728 0.36925812]
Mean: 0.3650388993607559
Standard Deviaton: 0.0016277940660940331


In [85]:
# Usamos tambien los datos con otro modelo 
# modelo DecisionTree
from sklearn.tree import DecisionTreeRegressor

# nombramos una variable para el modelo
tree_reg = DecisionTreeRegressor(random_state=42)

# Definir el pipeline: Transformacion + modelo
model_tree_reg = Pipeline(steps=[('transformer', transformer), ('tree_reg', tree_reg)])

# Ajustamos el modelo
model_tree_reg.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['sqfeet', 'beds', 'baths',
                                                   'cats_allowed',
                                                   'dogs_allowed',
                                                   'smoking_allowed',
                                                   'wheelchair_access',
                                                   'electric_vehicle_charge',
                                                   'comes_furnished']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['region', 'type',
                                                   'laundry_options',
                                                   'parking_options',
                                                   'state'])])),
                ('tree_reg', DecisionTreeRegressor(random_state=42))])

In [37]:
# score de validacion cruzada cv=10
tree_reg_scores = cross_val_score(model_tree_reg, X, y, scoring='neg_mean_squared_error', cv=10)


In [38]:
# cambio de signo y sacar raiz
tree_rmse_scores =np.sqrt(-tree_reg_scores)

# mostrar el resultado usando la funcion display_scores
display_scores(tree_rmse_scores)

Scores: [0.25254434 0.25190955 0.24857474 0.25045428 0.25065727 0.24937591
 0.25244781 0.25048284 0.25243759 0.2522515 ]
Mean: 0.25111358223258495
Standard Deviaton: 0.0013421850823743


In [106]:
# usamos el random forest como ultimo modelo para ver que modelo tomara la prediccion de datos
# random forest
from sklearn.ensemble import RandomForestRegressor

In [107]:
# nombramos variable para el modelo
forest_reg = RandomForestRegressor(n_estimators=10,random_state=42)

In [108]:
# definir pipeline: transformacion + modelo
model_forest_reg = Pipeline(steps=[('transformer', transformer), ('forest_reg', forest_reg)])

In [109]:
# ajustar el modelo
model_forest_reg.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['sqfeet', 'beds', 'baths',
                                                   'cats_allowed',
                                                   'dogs_allowed',
                                                   'smoking_allowed',
                                                   'wheelchair_access',
                                                   'electric_vehicle_charge',
                                                   'comes_furnished']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['region', 'type',
                                                   'laundry_options',
                                                   'parking_options',
                                                   'state'])])),
                ('forest_reg',
                 RandomForestRegressor(n_estimators=10, random_state=42))])

In [110]:
print(model_forest_reg.score(X_test, y_test))

0.809100216450744


In [111]:
# evaluacion, calculamos el score con validacion cruzada
forest_reg_scores = cross_val_score(model_forest_reg, X, y, scoring='neg_mean_squared_error', cv=10)

In [112]:
forest_rmse_score =np.sqrt(-forest_reg_scores)

In [113]:
display_scores(forest_rmse_score)

Scores: [0.21547554 0.21569072 0.21302437 0.21605017 0.21535003 0.21364374
 0.21403084 0.2133177  0.21666791 0.21736311]
Mean: 0.21506141162132422
Standard Deviaton: 0.0014060244717823617


In [114]:
# corremos el testeo con los datos, observamos que segun la division estandard y la media el modelo forest nos arroja mejores resultados
y_predict = model_forest_reg.predict(X_test)

In [115]:
# verificamos los datos predecidos, convertimos a lista para observar mejor los datos
list(y_predict)

[0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.6,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.9,
 1.0,
 1.0,
 1.0,
 0.616432150638033,
 1.0,
 1.0,
 0.4,
 1.0,
 0.4,
 0.0,
 0.0,
 0.8,
 1.0,
 0.0,
 0.0,
 1.0,
 0.2405960451459544,
 0.2,
 0.0,
 1.0,
 0.1,
 0.10588235294117647,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.64,
 1.0,
 0.4309061842894339,
 0.0,
 0.9,
 0.9,
 0.0,
 0.1,
 0.1,
 1.0,
 1.0,
 0.9,
 1.0,
 0.0,
 0.0,
 0.0,
 0.1,
 0.0,
 0.9,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.1,
 0.0,
 0.1,
 0.1,
 1.0,
 0.64,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.7,
 1.0,
 0.0,
 1.0,
 1.0,
 0.1,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.50625,
 0.0,
 0.6,
 1.0,
 0.0,
 0.0,
 0.3,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9,
 0.0,
 0.0,
 0.0,
 0.06761904761904762,
 1.0,
 0.0,
 0.0,
 0.0,
 0.2,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.6698245614035088,
 0.0,
 1.

In [36]:
list(y_test)

[0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,


In [37]:
# Afinamos el modelo randomforest
# utilizamos la funcion grindsearchcv para encontrar los valores optimos
from sklearn.model_selection import GridSearchCV

In [38]:
# definicion de posibles parametros
n_estimators = [3, 10, 30, 50, 80, 100]
min_samples_split = [2, 4, 6, 8]
max_depth = [2, 4, 6, 8, 10]

params = { 'forest_reg__n_estimators':n_estimators, 'forest_reg__min_samples_split':min_samples_split, 'forest_reg__max_depth':max_depth}
print(params)

{'forest_reg__n_estimators': [3, 10, 30, 50, 80, 100], 'forest_reg__min_samples_split': [2, 4, 6, 8], 'forest_reg__max_depth': [2, 4, 6, 8, 10]}


In [39]:
# definicion del grid de busqueda
grid = GridSearchCV(model_forest_reg, param_grid=params, cv=10, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

In [40]:
# ajuste del modelo usando todas las posibles combinaciones de parametros
grid.fit(X,y)

Fitting 10 folds for each of 120 candidates, totalling 1200 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['sqfeet',
                                                                          'beds',
                                                                          'baths',
                                                                          'cats_allowed',
                                                                          'dogs_allowed',
                                                                          'smoking_allowed',
                                                                          'wheelchair_access',
                                                                          'electric_vehicle_charge',
                                                                          'comes_furnished']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder())]),
                                                                         ['region',
                                                                          'type',
                                                                          'laundry_options',
                                                                          'parking_options',
                                                                          'state'])])),
                                       ('forest_reg',
                                        RandomForestRegressor(n_estimators=10,
                                                              random_state=42))]),
             n_jobs=-1,
             param_grid={'forest_reg__max_depth': [2, 4, 6, 8, 10],
                         'forest_reg__min_samples_split': [2, 4, 6, 8],
                         'forest_reg__n_estimators': [3, 10, 30, 50, 80, 100]},
             scoring='neg_mean_squared_error', verbose=2)

In [41]:
# mostrar la mejor combinacion
grid.best_params_


{'forest_reg__max_depth': 10,
 'forest_reg__min_samples_split': 2,
 'forest_reg__n_estimators': 100}

In [42]:
# ajustar el modelo con los nuevos parametros
forest_reg = RandomForestRegressor(random_state=42, max_depth=10, min_samples_split=2, n_estimators=100)

In [43]:
# definir el pipeline : transformacion + modelo
model_forest_reg = Pipeline(steps=[('transformer', transformer), ('forest_reg', forest_reg)])

In [44]:
# ajustar al modelo
model_forest_reg.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['sqfeet', 'beds', 'baths',
                                                   'cats_allowed',
                                                   'dogs_allowed',
                                                   'smoking_allowed',
                                                   'wheelchair_access',
                                                   'electric_vehicle_charge',
                                                   'comes_furnished']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['region', 'type',
                                                   'laundry_options',
                                                   'parking_options',
                                                   'state'])])),
                ('forest_reg',
                 RandomForestRegressor(max_depth=10, random_state=42))])

In [46]:
# evaluacion, calculamos el score con la validacion cruzada cv=10
from sklearn.model_selection import cross_val_score
forest_reg_scores = cross_val_score(model_forest_reg, X, y, scoring='neg_mean_squared_error', cv=10)

In [47]:
# realizar el cambio de signo y sacar la raiz
forest_rms_scores = np.sqrt(-forest_reg_scores)

In [50]:
# mostrar el resultado usando la funcion display_scores
display_scores(forest_rms_scores)

Scores: [0.38078584 0.3802587  0.38095921 0.38431463 0.38086863 0.38309667
 0.38135415 0.38271375 0.38131212 0.38416555]
Mean: 0.38198292309327114
Standard Deviaton: 0.001397003720069659


In [89]:
y_pred = model_linear_regresion.predict(X_test)
y_pred

array([0.13817093, 0.13814569, 0.82895167, ..., 0.72610004, 0.82337996,
       0.15455372])

In [83]:
# Trabajamos el programa con el archivo de testeo
data_test = pd.read_parquet('test.parquet')

In [84]:
# limpiamos los datos del archivo de testeo
data_test['laundry_options'] = data_test['laundry_options'].replace({None: 'Sin dato'})
data_test['parking_options'] = data_test['parking_options'].replace({None: 'Sin dato'})

In [86]:
# seleccionamos las columnas a usar en el modelo de prediccion
X_testeo_archivo = data_test[['region', 'type', 'sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed', 'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'laundry_options', 'parking_options', 'state', 'url', 'region_url', 'image_url']]

In [99]:
# lo corremos el modelo 
y_pred = model_linear_regresion.predict(X_testeo_archivo)
y_pred

array([0.28157561, 0.32433623, 0.14945968, ..., 0.29846735, 0.77496755,
       0.44039681])

In [116]:
y_predict = model_forest_reg.predict(X_testeo_archivo)

In [117]:
# convertimos el modelo en una lista
prediccion = list(y_predict)

In [118]:
# realizamos la conversion a entero los elementos de la lista
my_formatted_list = [ '%.0f' % elem for elem in prediccion ]

In [119]:
# convertimos cada elemento de la lista a entero para poder sacar el valor absoluto
my_formatted_list = [int(x) for x in my_formatted_list]

In [120]:
# sacamos el valor absoluto por si hay un elemento negativo
res =  [abs(ele) for ele in my_formatted_list]

In [121]:
# lo convertimos en un df para poder exportar a csv la prediccion
predict_df = pd.DataFrame(res, columns=['pred'])
predict_df

,pred
0,0
1,0
2,0
3,0
4,0
...,...
38493,0
38494,1
38495,0
38496,0


In [122]:
# exportamos el df en un csv sin indice
predict_df.to_csv('valdez101.csv', index = False)